# Multi-Output

In [1]:
import math
import torch
import gpytorch
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# 导入数据
params = np.loadtxt('0823paramsnp.txt')
edpResults = np.loadtxt('0823edpResult.txt')
# edpResults[:, [3, 7]] = edpResults[:, [7, 3]]
# 转化为torch tensor
train_x = torch.from_numpy(params[:1600]).to(torch.float)
train_y = torch.from_numpy(edpResults[:1600, 3:7]).to(torch.float)

In [3]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=4
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=4, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=4)
model = MultitaskGPModel(train_x, train_y, likelihood)

In [4]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iterations = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    optimizer.step()

Iter 1/50 - Loss: 1.115
Iter 2/50 - Loss: 1.076
Iter 3/50 - Loss: 1.036
Iter 4/50 - Loss: 0.995
Iter 5/50 - Loss: 0.954
Iter 6/50 - Loss: 0.913
Iter 7/50 - Loss: 0.871
Iter 8/50 - Loss: 0.829
Iter 9/50 - Loss: 0.786
Iter 10/50 - Loss: 0.742
Iter 11/50 - Loss: 0.698
Iter 12/50 - Loss: 0.653
Iter 13/50 - Loss: 0.607
Iter 14/50 - Loss: 0.562
Iter 15/50 - Loss: 0.515
Iter 16/50 - Loss: 0.469
Iter 17/50 - Loss: 0.422
Iter 18/50 - Loss: 0.374
Iter 19/50 - Loss: 0.327
Iter 20/50 - Loss: 0.279
Iter 21/50 - Loss: 0.231
Iter 22/50 - Loss: 0.182
Iter 23/50 - Loss: 0.133
Iter 24/50 - Loss: 0.083
Iter 25/50 - Loss: 0.034
Iter 26/50 - Loss: -0.017
Iter 27/50 - Loss: -0.067
Iter 28/50 - Loss: -0.117
Iter 29/50 - Loss: -0.168
Iter 30/50 - Loss: -0.218
Iter 31/50 - Loss: -0.269
Iter 32/50 - Loss: -0.319
Iter 33/50 - Loss: -0.369
Iter 34/50 - Loss: -0.420
Iter 35/50 - Loss: -0.471
Iter 36/50 - Loss: -0.521
Iter 37/50 - Loss: -0.572
Iter 38/50 - Loss: -0.622
Iter 39/50 - Loss: -0.671
Iter 40/50 - Loss: -

In [5]:
# 转化为torch tensor
test_x = torch.from_numpy(params[1600:]).to(torch.float)
test_y = torch.from_numpy(edpResults[1600:, :3]).to(torch.float)

In [6]:
# Set into eval mode
model.eval()
likelihood.eval()

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions_test = likelihood(model(test_x))
    mean_test = predictions_test.mean

In [7]:
# Set into eval mode
model.eval()
likelihood.eval()

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions_train = likelihood(model(train_x))
    mean_train = predictions_train.mean

c:\ProgramData\miniconda3\envs\torchenv310\lib\site-packages\gpytorch\models\exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


In [8]:
from sklearn.metrics import r2_score
# 训练集
print('训练集：')
r_squared_sklearn_train1 = r2_score(edpResults[:1600, 3], mean_train[:, 0].numpy())
print('idr1:', r_squared_sklearn_train1)
r_squared_sklearn_train2 = r2_score(edpResults[:1600, 4], mean_train[:, 1].numpy())
print('idr2:', r_squared_sklearn_train2)
r_squared_sklearn_train3 = r2_score(edpResults[:1600, 5], mean_train[:, 2].numpy())
print('idr3:', r_squared_sklearn_train3)
r_squared_sklearn_train4 = r2_score(edpResults[:1600, 6], mean_train[:, 3].numpy())
print('ridr:', r_squared_sklearn_train4)
# r_squared_sklearn_train4 = r2_score(edpResults[:1600, 4], mean_train[:, 4].numpy())
# print('ridr:', r_squared_sklearn_train4)
# r_squared_sklearn_train4 = r2_score(edpResults[:1600, 5], mean_train[:, 5].numpy())
# print('ridr:', r_squared_sklearn_train4)
# r_squared_sklearn_train4 = r2_score(edpResults[:1600, 6], mean_train[:, 6].numpy())
# print('ridr:', r_squared_sklearn_train4)
# 测试集
print('测试集：')
r_squared_sklearn_test1 = r2_score(edpResults[1600:, 3], mean_test[:, 0].numpy())
print('idr1:', r_squared_sklearn_test1)
r_squared_sklearn_test2 = r2_score(edpResults[1600:, 4], mean_test[:, 1].numpy())
print('idr2:', r_squared_sklearn_test2)
r_squared_sklearn_test3 = r2_score(edpResults[1600:, 5], mean_test[:, 2].numpy())
print('idr3:', r_squared_sklearn_test3)
r_squared_sklearn_test4 = r2_score(edpResults[1600:, 6], mean_test[:, 3].numpy())
print('ridr:', r_squared_sklearn_test4)
# r_squared_sklearn_test2 = r2_score(edpResults[1600:, 4], mean_test[:, 4].numpy())
# print('idr2:', r_squared_sklearn_test2)
# r_squared_sklearn_test3 = r2_score(edpResults[1600:, 5], mean_test[:, 5].numpy())
# print('idr3:', r_squared_sklearn_test3)
# r_squared_sklearn_test4 = r2_score(edpResults[1600:, 6], mean_test[:, 6].numpy())
# print('ridr:', r_squared_sklearn_test4)

训练集：
idr1: 0.95659837014623
idr2: 0.8005295535966
idr3: 0.8781310005449208
ridr: 0.8790380352330991
测试集：
idr1: 0.9684621695815921
idr2: 0.7849087237375978
idr3: 0.8319429789658452
ridr: 0.8574230193771735
